# MOS Extragalactic Science Usecase
(Susan Kassin, Iva Momcheva, Sara Ogaz)

## Outline

For a given galaxy (let's say there are ~100 of them)


1. Inspect cut-out 2D spectra (reduced)
    * play with contrast
    * zoom in/out
2. Inpsect 2D spectra on full detection image (not reduced)
3. Inspect cut-out image (made from other image data of object)
    * Should making the cutout be part of this?
    * place position angle (PA) slits on image
    * play with contrast
    * zoom in/out
4. Co-add selected 2D cut-out spectra and plot/inspect
    * Shift and co-add to get right alignment (cross-correlate). Gloss over this step for this notebook
5. Examine morhphology of emission lines and compare to image.
    * align image such that slit is vertical

Example Question: Do star-forming clumps in galaxies show up in the emission lines?

### Notes
* Test steps 1-3, and 5 in ginga.
* Test step 3 using glue as well (Didn't get around to this). Placing slits on image may be challenging. Will try using astrowidgets for ginga (new/under **HEAVY DEV**).
* (Didn't end up using this) Install for astrowidgets is "pip install git+git://github.com/astropy/astrowidgets.git@master". For an example notebook on this astrowidgets see here: https://github.com/astropy/astrowidgets/blob/master/example_notebooks/ginga_widget.ipynb

* Install list: ginga

### Conclusions

* Not satisfied with current documentation for using ginga in a jupyter notebook. Even our team had trouble figuring out the right way to do this, and ended up using the server ginga approach, which has the ginga windows pop up in a new tab (not desired). Was not clear how to have ginga show up inside the notebook. Although this is essentially what astropywidgets is doing(wrappers around ginga for the notebook).

* Inline ginga window buggy, when you mouse on and then off the image, the notebook jumps down.

* A width=1000, height=120 in the glue inline did not pan very well, was slow to respond (on Sara's machine).

* Pey Lian was (sort of) able to get ginga to match coordinates between frames for panning.

* Would like to do image display maniputation with sliders etc.

### Made cutout in separate notebook, but here's the summary

In [ ]:
import astropy
from astropy.io import fits

image = fits.open('sxdsz2c_dr1.fits')
image_wcs = astropy.wcs.WCS(header = image[0].header)

pos = astropy.coordinates.SkyCoord(["02:18:09.440 -04:45:12.24"], frame=FK5, unit=(u.hourangle, u.deg))

cutout = astropy.nddata.utils.Cutout2D(image[0].data, pos, size=(1000,1000),wcs=image_wcs)

plt.imshow(cutout.data)
plt.show()

cutout_header = cutout.wcs.to_header()
hdu = fits.PrimaryHDU(cutout.data,header = cutout_header)

hdu.writeto('SXDSN-19813_cutout.fits')

# Ginga

## Setup ginga instances, one for cutout image, one for s2D file

In [1]:
import astropy
from astropy.io import fits

In [2]:
# Ginga setup instructions taken from https://gist.github.com/ejeschke/6067409
from ginga.web.pgw import ipg
# Set this to True if you have a non-buggy python OpenCv bindings--it greatly speeds up some operations
use_opencv = False

server = ipg.make_server(host='localhost', port=9914, use_opencv=use_opencv)

In [3]:
# Start viewer server
# IMPORTANT: if running in an IPython/Jupyter notebook, use the no_ioloop=True option
server.start(no_ioloop=True)

In [4]:
# Get a viewer
# This will get a handle to the viewer v1 = server.get_viewer('v1')
cutout = server.get_viewer('cutout')

In [5]:
# Get a viewer
# This will get a handle to the viewer v1 = server.get_viewer('v1')
s2d = server.get_viewer('s2d')

In [6]:
# where is my viewer
cutout.url

'http://localhost:9914/app?id=cutout'

In [7]:
# open the viewers in a new window
cutout.open()
s2d.open()

## Load images into ginga windows

In [8]:
# Load an image into the viewer
# (change the path to where you downloaded the sample images, or use your own)
cutout.load('SXDSN-19813_cutout.fits')

In [3]:
hdulist = fits.open("ift0043_2spec.fits")
s2d_hdu = hdulist[11]
#s2d.load_hdu(s2d_hdu)

In [10]:
# Scale example (can do it programtically, not sure if there's a keyboard shortcut)
s2d.set_autocut_params('zscale', contrast=0.90)

# Or, go to active plot window, press space bar, then press "t".  Was not clearly documented on this.

# Zoom example
# plus / negative key

# Move around image
# on track pad, use both fingers

## Overplot slit

In [11]:
# Overplot slit
canvas=cutout.add_canvas()
Box = canvas.get_draw_class('box')

In [12]:
# Assume we have slit center, x and y sizes in pixels, and poistion (rotation angle)
# This is glossing over a potentially large effor part of this workflow.
canvas.add(Box(x=500, y=500, xradius=4, yradius=34, rot_deg=30, color='yellow'))

'@1'

# Ginga inline

### Scale keyboard shortcut
Or, go to active plot window, press space bar, then press "t".  Was not clearly documented on this.

### Zoom keyboard shortcut
plus / negative key

### Move keyboard shortcut
on track pad, use both fingers

In [4]:
import ipywidgets
import ginga
from ginga.web.jupyterw.ImageViewJpw import EnhancedCanvasView

In [22]:
# Setup ginga display for multiple s2D images.

imgs = [ipywidgets.Image(format='jpeg', width=800, height=100) for i in range(3)]

vs=[]
hdus = [fits.open("ift0043_2spec.fits")[11].data[0], fits.open("ift0045_2spec.fits")[11].data[0], 
        fits.open("ift0048_2spec.fits")[11].data[0]]
for i, im in enumerate(imgs):
    v = EnhancedCanvasView()
    v.set_widget(im)
    v.get_bindings().enable_all(True)
    v.set_cmap(ginga.cmap.get_cmap('viridis'))
    v.load_data(hdus[i])
    v.set_scale_limits(0,1)
    vs.append(v)
    

box = ipywidgets.VBox(imgs)
box

In [26]:
# Setup single 
img2 = ipywidgets.Image(format='jpeg', width=500, height=500)

v2 = EnhancedCanvasView()
v2.set_widget(img2)
v2.get_bindings().enable_all(True)

#v1.set_cmap(ginga.cmap.get_cmap('viridis'))

#v1.load_data(d.data)
#v1.zoom_to(5)
#v1.load_hdu(s2d_hdu)
cut = fits.open("SXDSN-19813_cutout.fits")
v2.load_data(cut[0].data)

img2

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x03\x02\x0…

In [27]:
# Overplot slit
canvas=v2.add_canvas()
Box = canvas.get_draw_class('box')

In [28]:
# Assume we have slit center, x and y sizes in pixels, and poistion (rotation angle)
# This is glossing over a potentially large effor part of this workflow.
canvas.add(Box(x=500, y=500, xradius=4, yradius=34, rot_deg=30, color='yellow'))

'@1'